### Project for Application of Differential Privacy with PATE
<font size="3">
This project is to show the application of differential privacy (DP) with PATE (Private Aggregation of Teacher Ensembles) algorithm on MNIST handwritten digits dataset. You will get the intuition on the relationship between privacy loss and accuracy of differenial-privacy outputs. For details, please refer to README file. <font/>

In [1]:
# import necessary libraries
import os
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Subset

In [2]:
# Define a transform 
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Load train and test data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
print('Trainset size:', len(trainset))
print('Testset size:', len(testset))

Trainset size: 60000
Testset size: 10000


In [3]:
# Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)
model = Net()
print(model)       

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [4]:
# Define a device to use GPU 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

###  Training of Teacher Models and Prediction on Student Dataset 

In [5]:
def get_teacher_preds(num_teachers, num_examples, epochs): 
    
    # Make a new directory to contain prediction files for each teacher model              
    dirName = "teachers " + str(num_teachers)
    if not os.path.exists(dirName):
        os.mkdir(dirName)
        
    # Change directory to save the files in the created directory
    os.chdir(dirName) 
    
    # Train teacher models
    for i in range(num_teachers):
        train_idx = list(range(i * num_examples, (i+1) * num_examples))
        train = Subset(trainset, train_idx)
        trainloader = torch.utils.data.DataLoader(train, batch_size=64, num_workers=0)      
       
        model = Net()
        criterion = nn.NLLLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)        
        model.to(device)
        
        for e in range(epochs):
            running_loss = 0
            for images, labels in trainloader:                 
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()        
                output = model(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()        
                running_loss += loss.item() 
                
        # Print training loss for each teacher model     
        print("Teacher_" + str(i+1) + " training loss:", running_loss/len(trainloader))  
        
        # Define student dataset 
        student_idx = list(range(0, 6000))      
        student_data = Subset(testset, student_idx)  
        student_loader = torch.utils.data.DataLoader(student_data, batch_size=64, num_workers=0)
        
        # With the trained teacher models, make predictions on student dataset       
        model.to(device) 
        model.eval()
        
        outputs = torch.zeros(0, dtype=torch.long).to(device)
        for images, labels in student_loader:             
            images, labels = images.to(device), labels.to(device)                             
            output = model.forward(images)
            pred = torch.argmax(torch.exp(output), dim=1)
            outputs = torch.cat((outputs, pred))            
    
        # Save outputs as .pt file for use in case of kernel-restart, so that retraining is not needed.
        file_name = 'pred_' + str(i+1) + '.pt'  
        torch.save(outputs, file_name)          

In [6]:
# Execute the above function

num_teachers = 50
num_examples = len(trainset)//num_teachers
epochs = 10

get_teacher_preds(num_teachers, num_examples, epochs)

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Teacher_1 training loss: 0.5744781086319372
Teacher_2 training loss: 0.41659486293792725
Teacher_3 training loss: 0.560007454533326
Teacher_4 training loss: 0.5285757889873103
Teacher_5 training loss: 0.568955637906727
Teacher_6 training loss: 0.4504693346588235
Teacher_7 training loss: 0.6150174360526236
Teacher_8 training loss: 0.6178547834095202
Teacher_9 training loss: 0.5097571485920956
Teacher_10 training loss: 0.5873075943244132
Teacher_11 training loss: 0.6334171561818374
Teacher_12 training loss: 0.5250420617429834
Teacher_13 training loss: 0.5332103591216238
Teacher_14 training loss: 0.6178819599904513
Teacher_15 training loss: 0.6038057584511606
Teacher_16 training loss: 0.4882370766840483
Teacher_17 training loss: 0.5581442792164651
Teacher_18 training loss: 0.5499839453320754
Teacher_19 training loss: 0.5141241487703825
Teacher_20 training loss: 0.481162965297699
Teacher_21 training loss: 0.596268609950417
Teacher_22 training loss: 0.48046356439590454
Teacher_23 training l

In [7]:
# Check current working directory to load prediction files
!pwd

/c/Users/USER/private-ai-master


In [8]:
# In case current working directory is not 'teachers 50', uncomment the command below and run the command
# os.chdir("teachers 50")

In [9]:
# Make sure that current working directory is 'teachers 50'
!pwd

/c/Users/USER/private-ai-master/teachers 50


### Load prediction files generated in the training process and merge loaded files

In [10]:
num_teachers = 50

preds = []
for i in range(num_teachers): 
    file_name = 'pred_' + str(i+1) + '.pt'
    pred = torch.load(file_name).cpu().numpy()
    preds.append(pred)    
    teacher_preds = np.vstack((preds))
print(teacher_preds.shape)

(50, 6000)


### Generate DP- aggregated labels 

In [11]:
def aggragate_teacher_preds(epsilon):  
    
    labels = np.array([]).astype(int)
    for pred in np.transpose(teacher_preds):   
        label_counts = np.bincount(pred, minlength=10)    
        beta = 1 / epsilon

        for i in range(len(label_counts)):
            label_counts[i] += np.random.laplace(0, beta, 1)
        
        new_label = np.argmax(label_counts)   
        labels = np.append(labels, new_label) 
        labels =  torch.from_numpy(labels)      
   
    return labels

## PATE Analysis

In [12]:
from syft.frameworks.torch.differential_privacy import pate

C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
num_teachers, num_examples, num_labels = (50, 6000, 10)
diff_priv_labels = aggragate_teacher_preds(0.2)  # true answers

In [14]:
# Perform PATE Analysis
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=teacher_preds, indices=diff_priv_labels, noise_eps=0.2, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 971.5129254649704
Data Dependent Epsilon: 106.92126000063695


### Train a new model on labeled student dataset . This will be the final "DP" model.<br/>

In [15]:
# Redefine student train data with aggragated labels  
testset.data[:6000] = torch.FloatTensor(testset.data.clone().detach().numpy()[:6000])
testset.targets[:6000] = diff_priv_labels

student_train = Subset(testset, list(range(6000)))
student_trainloader = torch.utils.data.DataLoader(student_train, batch_size=64, num_workers=0)

model_2 = Net()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_2.parameters(), lr=0.001)
model_2.to(device)
epochs = 15

for e in range(epochs): 
    train_loss = 0.0    
    for images, labels in student_trainloader:          
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()        
        output = model_2(images)        
        loss = criterion(output, labels)        
        loss.backward()       
        optimizer.step()        
        train_loss += loss.item()
            
    print("Epoch: {}/{}   ".format(e+1, epochs),
                      "Training Loss: {:.5f}   ".format(train_loss/len(student_trainloader)))  

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/15    Training Loss: 1.54432   
Epoch: 2/15    Training Loss: 0.58701   
Epoch: 3/15    Training Loss: 0.44037   
Epoch: 4/15    Training Loss: 0.38858   
Epoch: 5/15    Training Loss: 0.34863   
Epoch: 6/15    Training Loss: 0.31705   
Epoch: 7/15    Training Loss: 0.30802   
Epoch: 8/15    Training Loss: 0.28909   
Epoch: 9/15    Training Loss: 0.27849   
Epoch: 10/15    Training Loss: 0.26019   
Epoch: 11/15    Training Loss: 0.26151   
Epoch: 12/15    Training Loss: 0.25292   
Epoch: 13/15    Training Loss: 0.26207   
Epoch: 14/15    Training Loss: 0.24103   
Epoch: 15/15    Training Loss: 0.23573   


###  With the student model, make predictions on student test-data and calculate accuracy

In [16]:
student_test = Subset(testset, list(range(6000, 10000)))  
student_testloader = torch.utils.data.DataLoader(student_test, batch_size=64, num_workers=0)

test_loss = 0
accuracy = 0
model_2.to(device)
model_2.eval()

with torch.no_grad(): 
    for images, labels in student_testloader:              
        images, labels = images.to(device), labels.to(device)
        output = model_2(images)
        test_loss += criterion(output, labels).item()        
               
        # Calculate test accuracy
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)        
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

print("Test Loss: {:.5f}   ".format(test_loss/len(student_testloader)),
      "Test Accuracy: {:.5f}".format(accuracy/len(student_testloader)))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Loss: 0.17437    Test Accuracy: 0.94940
